In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import plotly.express as px
import networkx as nx

In [60]:
T = np.array([
    [0, 1/2, 1/2],
    [1, 0, 0],
    [0, 1, 0]
])

In [61]:
eig = sp.linalg.eig(T, left=True, right=False)
eig_vals, eig_vecs = eig
index = np.argwhere(np.isclose(eig_vals, 1))
eig_vec_1 = eig_vecs[:, index].flatten() / np.sum(eig_vecs[:, index])
eig_vec_1 = np.real_if_close(eig_vec_1)
eig_vec_1

array([0.4, 0.4, 0.2])

In [90]:
initial_belief = np.array([1, 0, 0])
correct_belief = 0.5

In [91]:
convergence = eig_vec_1 @ initial_belief
error =  np.square(convergence - correct_belief)
error

0.00999999999999994

In [254]:
dims = 6
A = np.zeros((dims, dims**2))
for i in range(dims):
    A[i, i*dims:(i+1)*dims] = 1

In [255]:
linear_constraint = sp.optimize.LinearConstraint(A, 1, 1)

In [256]:
bounds = sp.optimize.Bounds(0, 1)

In [257]:
def degroot_error(x, ibelief, cbelief, niter):
    dim = np.sqrt(len(x)).astype(int)
    m = x.reshape(dim, dim)
    m = m / np.sum(m, axis=1)
    mt = np.linalg.matrix_power(m, niter)
    pt = mt @ ibelief
    error =  np.sum(np.square(pt - cbelief))
    return error

In [258]:
initial_belief = np.array([1, 0, 0, 1, 1, 0])
correct_belief = 1
num_iterations = 100

In [263]:
n_trials = 100
initial_conditions = []
for i in range(n_trials):
    x = np.random.rand(dims, dims)
    x = x / np.sum(x, axis=1)
    x = x.flatten()
    initial_conditions.append(x)
    
results = []
for x in initial_conditions:
    res = sp.optimize.minimize(fun=degroot_error, args=(initial_belief, correct_belief, num_iterations), x0=x, method='trust-constr', constraints=linear_constraint, bounds=bounds, options={'maxiter':  10000})
    results.append(res)

In [264]:
meta_data = []
solutions = []
for r in results:
    meta_data.append({'fun': r.fun, 'iter': r.nit, 'success': r.success})
    solutions.append(r.x)
                 
meta_data = pd.DataFrame(meta_data)

In [265]:
dist = sp.spatial.distance.pdist(solutions, metric='euclidean')
hist, bins = np.histogram(dist, bins=10)
px.bar(x=bins[1:], y=hist)

In [267]:
meta_data

,fun,iter,success
0,2.562898e-07,35,True
1,3.071001e-07,32,True
2,3.058851e-07,30,True
3,5.042834e-07,28,True
4,6.873908e-07,28,True
...,...,...,...
95,3.062381e-07,32,True
96,5.036776e-07,30,True
97,6.841337e-07,30,True
98,5.025125e-07,29,True


## Myopic Search

In [202]:
def degroot_err(m, ibelief, cbelief, niter):
    m = m / np.sum(m, axis=1)
    mt = np.linalg.matrix_power(m, niter)
    pt = mt @ ibelief
    error =  np.sum(np.square(pt - cbelief))
    return error

In [239]:
class Result:
    def __init__(self, m, error, initial_error, converged):
        self.m = m
        self.error = error
        self.initial_error = initial_error
        self.converged = converged

def myopic_search(m0, niter_search, ibelief, cbelief, niter_degroot, convergence_bar):
    m = m0.copy()
    initial_error = degroot_err(m, ibelief, cbelief, niter_degroot)
    error = initial_error.copy()
    n_rounds_no_improvement = 0
    converged = False
    for i in range(niter_search):
        indices_non_zero = np.argwhere(m > 0)
        
        if len(indices_non_zero) == 0:
            continue
            
        index_non_zero = np.random.choice(np.arange(len(indices_non_zero)))
        i, j = indices_non_zero[index_non_zero]
        
        indices_zero = np.argwhere(m[i] == 0).flatten()
        
        if len(indices_zero) == 1:
            continue
            
        indices_zero = indices_zero[indices_zero != i]
        k = np.random.choice(indices_zero)
        m1 = m.copy()
        
        m1[i, j] = 0
        m1[i, k] = 1
        error1 = degroot_err(m1, ibelief, cbelief, niter_degroot)
        
        if error1 < error:
            m = m1
            error = error1
            n_rounds_no_improvement = 0
        else:
            n_rounds_no_improvement += 1
            
        if n_rounds_no_improvement > convergence_bar:
            converged = True
            break

    return Result(m, error, initial_error, converged)

In [267]:
dims = 8
p = 0.5
init_belief = np.random.rand(dims)
correct_belief = 0.7
num_iterations = 1000
init_belief

array([0.39660715, 0.45974194, 0.70609609, 0.62631433, 0.64101558,
       0.28223579, 0.34561588, 0.69964285])

In [280]:
n_trials = 100
initial_conditions = []
for i in range(n_trials):
    m = nx.adjacency_matrix(nx.erdos_renyi_graph(n=dims, p=0.5)).todense()
    if np.min(np.sum(m, axis=1)) == 0:
        continue
    initial_conditions.append(m)

unique_initial_conditions = np.unique([s.flatten() for s in initial_conditions], axis=0)
unique_initial_conditions = [s.reshape(dims, dims) for s in unique_initial_conditions]
len(unique_initial_conditions)

96

In [287]:
initial_errors = [degroot_err(x, init_belief, correct_belief, num_iterations) for x in unique_initial_conditions]
sorted(initial_errors)[::-1]

[1.193673335298378,
 0.920294277019714,
 0.8602842398365922,
 0.8002742026534697,
 0.7422479683524261,
 0.6922561357238364,
 0.6708310645973035,
 0.6602507825595114,
 0.6569849301957983,
 0.6569849301957711,
 0.6314959730759435,
 0.620244091104091,
 0.6202440911040824,
 0.5994141608421875,
 0.5994141608421873,
 0.5994141608421768,
 0.5688069163757032,
 0.5578336524336425,
 0.5578336524336425,
 0.5578336524336076,
 0.554197541304896,
 0.5302290353294266,
 0.5279972570870899,
 0.5247179094656295,
 0.5247179094656291,
 0.5247179094656275,
 0.524717909465624,
 0.5218193242363095,
 0.5218193242362865,
 0.5218193242362632,
 0.519427228636463,
 0.5194272286364598,
 0.5194272286364205,
 0.5194272286364024,
 0.5002240167378476,
 0.5002240167378442,
 0.5002240167378419,
 0.5002240167378331,
 0.49357470513859986,
 0.4919181292384494,
 0.49191812923843836,
 0.4810208048392067,
 0.4810208048392036,
 0.48062890255559265,
 0.480628902555584,
 0.4735528891008959,
 0.4627177434983755,
 0.46201693424065

In [282]:
results = []
for x in unique_initial_conditions:
    res = myopic_search(x, 1000, init_belief, correct_belief, num_iterations, 20)
    results.append(res)

In [284]:
meta_data = []
solutions = []
for r in results:
    meta_data.append({'error': r.error, 'initial_error': r.initial_error, 'converged': r.converged})
    solutions.append(r.m)
    print(r.m)

[[0 0 0 0 0 0 1 0]
 [0 0 0 1 1 0 1 0]
 [0 0 0 1 1 1 0 0]
 [1 1 1 0 1 0 0 1]
 [1 1 0 1 0 0 0 1]
 [0 1 1 1 1 0 0 1]
 [1 1 1 0 1 0 0 1]
 [0 0 1 1 0 1 1 0]]
[[0 0 1 1 1 0 0 0]
 [0 0 1 1 1 1 0 0]
 [0 1 0 1 1 0 0 0]
 [0 0 1 0 1 0 1 1]
 [1 0 1 0 0 0 0 1]
 [0 1 1 0 1 0 0 1]
 [0 0 1 1 1 1 0 1]
 [1 1 1 1 1 1 0 0]]
[[0 0 0 1 1 0 0 0]
 [0 0 1 1 1 0 0 0]
 [1 0 0 1 0 0 1 0]
 [1 0 0 0 1 0 0 1]
 [1 0 1 1 0 0 0 0]
 [1 0 1 1 0 0 0 0]
 [1 0 1 1 1 0 0 1]
 [1 1 0 1 1 0 0 0]]
[[0 0 1 0 1 0 0 0]
 [1 0 1 0 1 0 0 1]
 [0 1 0 1 1 0 0 0]
 [1 0 1 0 1 0 1 1]
 [1 1 0 1 0 0 0 1]
 [1 1 1 0 1 0 0 1]
 [1 0 1 1 0 0 0 1]
 [1 0 0 1 1 0 0 0]]
[[0 0 1 0 0 1 0 0]
 [0 0 1 1 0 1 1 1]
 [0 0 0 0 0 1 0 0]
 [0 1 0 0 1 0 1 1]
 [0 0 0 1 0 1 1 0]
 [1 1 0 0 0 0 1 1]
 [1 0 0 1 0 0 0 0]
 [0 1 0 1 0 0 1 0]]
[[0 0 0 0 1 1 0 1]
 [0 0 0 1 1 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 1 1 0 1 0 1 1]
 [1 1 0 1 0 1 1 1]
 [1 1 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0]
 [1 0 0 1 1 0 0 0]]
[[0 0 0 0 1 0 1 1]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [1 0 0 0 1 0 0 1]
 [1 1 

In [285]:
meta_data = pd.DataFrame(meta_data)
meta_data

,error,initial_error,converged
0,0.420063,0.519427,True
1,0.230597,0.395206,True
2,0.239692,0.400799,True
3,0.245020,0.404208,True
4,0.560003,0.620244,True
...,...,...,...
91,0.184474,0.366868,True
92,0.423462,0.521819,True
93,0.379776,0.491918,True
94,0.382200,0.493575,True


In [279]:
len(np.unique([s.flatten() for s in solutions], axis=0))

96